In [2]:
import tensorflow as tf
from datetime import datetime, timedelta
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from IPython.display import clear_output
import os
import mysql.connector
import glob
import traceback

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from IPython.display import clear_output


2025-06-24 09:44:43.557523: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
#sample run for time aware models.

In [1]:
def data_maker_pom_test (i):
    df = pd.read_csv(i, index_col = [0])
    df = df.iloc[df[df.timestamp.str.contains(df.timestamp.apply(lambda x: x.split(' ')[0]).unique()[-360])].iloc[0].name:]
    df.reset_index(drop = True, inplace = True)
    times = pd.DataFrame(df['timestamp'])
    pl = pd.DataFrame(df['pl'])
    dff = pd.DataFrame()
    st = datetime.strptime(df.timestamp.iloc[ 0 ], '%Y-%m-%d %H:%M:%S') #+ timedelta(days = 1)
    et = datetime.strptime(df.timestamp.iloc[-1 ], '%Y-%m-%d %H:%M:%S')
    while True:
        if st > et + timedelta(days = 1):
            break
        srt = datetime.strftime(st , '%Y-%m-%d %H:%M:%S')
        tmp = df[df['timestamp'].str.contains(srt.split(' ')[0])].copy()
        tmp.drop(columns = ['timestamp'], inplace = True)
        lis = list(tmp.columns)
        if len(tmp) == 0:
            st += timedelta(days = 1)
            continue
        scaler = StandardScaler()
        scaler.fit(tmp)
        tmp = scaler.transform(tmp)
        
        tmp = pd.DataFrame(tmp, columns = lis)
        dff = pd.concat([
            dff,
            tmp
        ])
        del scaler
        st += timedelta(days = 1)
    dff.reset_index(drop = True, inplace = True)
    dff['pl'] = pl['pl']
    return dff,times,df

def prepare_data_com_pom(df,i,times,timesteps):
    pl = df[['pl']]
    df['class'] = [1 if j[0] >0 else 0 for j in pl.values]
    y = df[['class']]
    df['pl'] = pl
    df['class'] = y
    col = list(df.columns)
    tdf = pd.DataFrame(columns = ['timestamp'])
    
    dt = times.iloc[0].values[0]
    dt = dt.split(' ')[0]
    et = times.iloc[-1].values[0]
    et = et.split(' ')[0]
    dtt = datetime.strptime(dt, '%Y-%m-%d')
    ett = datetime.strptime(et, '%Y-%m-%d')
    fd = pd.DataFrame(columns = col)
    fdd = df.copy()
    while dtt < ett + timedelta(days = 1):
        
        try:
            ind = times[times['timestamp'].str.contains(dtt.strftime('%Y-%m-%d'))].iloc[-1].name
            tdf = pd.concat([
                tdf,
                pd.DataFrame(
                    times[
                        times['timestamp'].str.contains(
                            dtt.strftime('%Y-%m-%d')
                        )].iloc[-1]
                ).T
            ])
        except:
            dtt+= timedelta(days = 1)
            continue 
        ad = pd.DataFrame(fdd.loc[ind-timesteps:ind],columns = col)
        ad = ad[col]
        fd = pd.concat([
            fd,
            ad
        ])
        dtt += timedelta(days = 1)
    fd.reset_index(drop = True, inplace = True)
    col = fd.columns
    return fd, col,tdf

def plc_pom(fd):
    data = fd['pl']
    range_50 = np.percentile(data, [16, 84]) 
    range_75 = np.percentile(data, [3, 97])  
    range_100 = np.percentile(data, [0, 100])
    fd['plc'] = fd['pl'].apply(
        lambda x : 1 if (x <= range_50[1] and x > 0 ) else( 
            -1
            if (x > range_50[0] and x < 0) else(
                2
                if (x > range_50[1] and x < range_75[1]) else(
                    -2
                    if (x < range_50[0] and x > range_75[0]) else (
                        3 if (x > range_75[1]) else(
                            -3 if (x < range_75[0]) else 0
                        )
                    )
                )
            )
        )
    )
    return fd, (range_50,range_75,range_100)

def plc_pom_4(fd):
    data = fd['pl']
    range_45 = np.percentile(data, [27.5, 72.5])
    range_50 = np.percentile(data, [16, 84]) 
    range_75 = np.percentile(data, [3, 97])  
    range_100 = np.percentile(data, [0, 100])
    fd['plc'] = fd['pl'].apply(
        lambda x : 1 if (x > 0 and x <= range_45[1]) else(
            -1 if(x<0 and x > range_45[0] ) else(
            2 if(x >= range_45[1] and x < range_50[1]) else(
            -2 if (x <= range_45[0] and x > range_50[0]) else(
            3 if (x >= range_50[1] and x < range_75[1]) else(
            -3 if (x <= range_50[0] and x > range_75[0]) else(
            4 if (x >= range_75[1]) else (
            -4 if (x <= range_75[0]) else 0
            )
            )
            )
            )
            )
        )
    ))
    return fd, (range_50,range_75,range_100)

def splitter_com(df,timesteps):
    xx = np.empty((0,timesteps + 1,len(df.columns) - 3))
    yy = np.empty((0,timesteps + 1 ,1))
    pll = np.empty((0,timesteps + 1 , 1))
    plcc = np.empty((0,timesteps + 1 , 1))
    
    yt = df[['class']]
    pltt = df[['pl']]
    pltc = df[['plc']]
    
    loop = len(df) / (timesteps+1)
    for i in range(0,int(loop)):
        x = df.loc[i*(timesteps+1): ((i+1) *(timesteps + 1 )) - 1].copy()
        y = yt.loc[i*(timesteps+1): ((i+1) *(timesteps + 1 )) - 1]
        pl = pltt.loc[i*(timesteps+1): ((i+1) *(timesteps + 1 )) - 1]
        plc = pltc.loc[i*(timesteps+1): ((i+1) *(timesteps + 1 )) - 1]
        
        x.drop(columns = ['class','pl','plc'], inplace = True)
        x = np.array(x).reshape(1,timesteps+1,len(x.columns))
        y = np.array(y).reshape(1,timesteps+1,1)
        pl = np.array(pl).reshape(1,timesteps+1,1)
        plc = np.array(plc).reshape(1,timesteps+1,1)
        
        xx = np.append(xx,x,axis = 0)
        yy = np.append(yy,y, axis = 0)
        pll = np.append(pll,pl, axis = 0)
        plcc = np.append(plcc,plc, axis = 0)
    return xx,yy,pll,plcc

def padder_com(x,y,pl,plc,sli,timesteps):
    xtt = np.empty((0,timesteps+1,x.shape[2]))
    ytt = np.empty((0,timesteps+1,y.shape[2]))
    plttt = np.empty((0,timesteps+1,pl.shape[2]))
    pltcc = np.empty((0,timesteps+1,plc.shape[2]))
    xee = np.empty((0,timesteps+1,x.shape[2]))
    yee = np.empty((0,timesteps+1,y.shape[2]))
    plee = np.empty((0,timesteps+1,pl.shape[2]))
    plecc = np.empty((0,timesteps+1,plc.shape[2]))
    for i in range(0,len(x) - sli):
        for j in range(1,timesteps+2):
            xt = np.pad(
                x[i][- j: ].reshape(1,j,x.shape[2]),
                ((0, 0), (timesteps + 1 -j, 0), (0, 0)),
                mode = 'constant',
                constant_values=-1
            )
            yt = np.pad(
                y[i][- j: ].reshape(1,j,y.shape[2]),
                ((0, 0), (timesteps + 1-j, 0), (0, 0)),
                mode = 'constant', 
                constant_values=-1
            )
            pltt = np.pad(
                pl[i][- j: ].reshape(1,j,pl.shape[2]),
                ((0, 0), (timesteps + 1 - j , 0), (0, 0)),
                mode = 'constant', 
                constant_values=0
            )
            pltc = np.pad(
                plc[i][- j: ].reshape(1,j,plc.shape[2]),
                ((0, 0), (timesteps + 1 - j , 0), (0, 0)),
                mode = 'constant', 
                constant_values=0
            )
            xtt = np.append(xtt,xt, axis = 0)
            ytt = np.append(ytt,yt, axis = 0)
            plttt = np.append(plttt,pltt, axis = 0)
            pltcc = np.append(pltcc,pltc, axis = 0)
    for i in range(len(x) - sli, len(x)):
        for j in range(1,timesteps+2):
            xt = np.pad(
                x[i][- j: ].reshape(1,j,x.shape[2]),
                ((0, 0), (timesteps + 1 - j, 0), (0, 0)),
                mode = 'constant', 
                constant_values=-1
            )
            yt = np.pad(
                y[i][- j: ].reshape(1,j,y.shape[2]),
                ((0, 0), (timesteps + 1 - j, 0), (0, 0)),
                mode = 'constant', 
                constant_values=-1
            )
            pltt = np.pad(
                pl[i][- j: ].reshape(1,j,pl.shape[2]),
                ((0, 0), (timesteps + 1 - j, 0), (0, 0)),
                mode = 'constant', 
                constant_values=0
            )
            pltc = np.pad(
                plc[i][- j: ].reshape(1,j,plc.shape[2]),
                ((0, 0), (timesteps + 1 - j, 0), (0, 0)),
                mode = 'constant', 
                constant_values=0
            )
            xee = np.append(xee,xt, axis = 0)
            yee = np.append(yee,yt, axis = 0)
            plee = np.append(plee,pltt, axis = 0)
            plecc = np.append(plecc,pltc, axis = 0)

    return xtt,ytt,plttt, pltcc,xee,yee,plee,plecc


In [6]:
import pandas as pd
df_main = pd.read_csv('../../data/market_indices.csv', index_col = [0])
df_main.head(3)

,symbol
0,CNXFMCG
1,XBANK
2,NIFTYQUALITY30


In [9]:
%%time
import gc

class StopAtAccuracy(tf.keras.callbacks.Callback):
    def __init__(self, target=0.995):
        super().__init__()
        self.target = target

    def on_epoch_end(self, epoch, logs=None):
        acc = logs.get("accuracy")  # Use "val_accuracy" if you're using validation accuracy
        if acc is not None and acc >= self.target:
            print(f"\nDesired accuracy reached. Stopping training.")
            self.model.stop_training = True
            
def run_simulation(i):
    try:
        timesteps = 3 #4
        sli = 108
        sym = '../data/Database_sh_backup/' + i + '.csv'

        dff,times,df = data_maker_pom_test (sym)


        fd, col , tdf = prepare_data_com_pom(dff.copy(),sym,times,timesteps)
        scaler = StandardScaler()
        scaler.fit(fd[['pl']][:len(fd) - sli])
        fd['pl'] = scaler.transform(fd[['pl']])
        fd,tup = plc_pom(fd)
        x,y,pl,plc = splitter_com(fd,timesteps)
        xt, yt, pltt, pltc,xe, ye, ple, plec = padder_com(x,y,pl,plc,sli,timesteps)
        otr = np.array([ [1 if i[0] == 1 else 0 ,0 if i[0] == 1 else 1]  for i in yt[:,-1,:]])
        ote = np.array([ [1 if i[0] == 1 else 0 ,0 if i[0] == 1 else 1]  for i in ye[:,-1,:]])
        otrc = np.array(
        [
            [
                1 if i[0] == -3 else 0 ,
                1 if i[0] == -2 else 0 ,
                1 if i[0] == -1 else 0 ,
                1 if i[0] == 1 else 0 ,
                1 if i[0] == 2 else 0 ,
                1 if i[0] == 3 else 0
            ]  for i in pltc[:,-1,:]
        ]
        )
        otec = np.array(
        [
            [
                1 if i[0] == -3 else 0 ,
                1 if i[0] == -2 else 0 ,
                1 if i[0] == -1 else 0 ,
                1 if i[0] == 1 else 0 ,
                1 if i[0] == 2 else 0 ,
                1 if i[0] == 3 else 0
            ]  for i in plec[:,-1,:]
        ]
        )

        model = Sequential()

        
        model.add(LSTM(128, input_shape=(3, 162), return_sequences=False))
        model.add(Dropout(0.3))

        model.add(Dense(6, activation='softmax'))  # Softmax for multiclass

        model.compile(optimizer=Adam(learning_rate=0.001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        timess = 400
        model.fit(
            xt[:, 1:, :], otrc[:],
            epochs=timess,
            callbacks=[StopAtAccuracy(target=0.995)]
        )

        yp = model.predict(xe[:, 1:, :])
        model.save("../cmg_results/LSTM/models/"+i+'.h5')
        del model
        gc.collect()
        count = 0
        li = []
        for ii in yp:
            if (count+1)%4 == 0:
                if (ii[0] + ii[1] + ii[2])>  (ii[3]+ii[4] + ii[5]):
                    li.append(0)
                else:
                    li.append(1)
            count += 1

        y_pred = pd.DataFrame(li, columns = ['pred'])
        y_pred['timestamp'] = list(list ( tdf[-int(len(yp)/4):]['timestamp'].apply(lambda x:x.split(' ')[0])))
        y_pred = y_pred [['timestamp', 'pred']]

# ================================================
# ⚠️ OHLC Data Usage (Commented Out)
# ================================================
# This section of the code utilizes OHLC (Open, High, Low, Close) financial market data.
# Due to licensing restrictions, the actual OHLC dataset used in this project cannot be
# uploaded or distributed publicly via this repository.
#
# Many OHLC data sources (e.g., Yahoo Finance, Bloomberg, NSE, etc.) allow personal or
# research use, but prohibit redistribution or public sharing of the data files.
#
# To reproduce results:
# - You may use your own OHLC data.
# - Or modify the script to fetch data from APIs (e.g., yfinance).
#
# =================================================
#         ori = '../data/Database_backup/'
#         ori_df = pd.read_csv(ori, index_col = [0])    
#         dt_end = datetime.strptime(y_pred.iloc[-1]['timestamp'],'%Y-%m-%d')
#         acc = pd.DataFrame(columns = ['date','accuracy', 'potential'])
#         for ind , row in y_pred.iterrows():
#             dt = datetime.strptime(row.timestamp,'%Y-%m-%d')
#             dt_orig = dt
#             close_previous = ori_df [ori_df.datetime.str.contains(dt_orig.strftime('%Y-%m-%d'))].iloc[-1].close
#             checker = 0
#             while True:
#                 dt += timedelta(days = 1)
#                 if dt > dt_end:
#                     checker = 1
#                     break
#                 sd = ori_df[ori_df.datetime.str.contains(dt.strftime('%Y-%m-%d'))].copy()
#                 if len(sd) > 0:
#                     break
#             if checker == 1:
#                 continue
#             if len(sd) > 0:
#                 sdd = sd[:int(len(sd) / 4)] #first quarter
#                 check = 0
#                 tmp = -1
#                 for indd, roww in sdd.iterrows():
#                     if row.pred == 1:
#                         if roww.close > close_previous:
#                             if tmp < (roww.close - close_previous):
#                                 tmp = roww.close - close_previous
#                             check = 1
#                     else:
#                         if roww.close < close_previous:
#                             if tmp < (close_previous - roww.close):
#                                 tmp = close_previous - roww.close
#                             check = 1
#                 if check == 1:
#                     acc = pd.concat([acc, pd.DataFrame(
#                         [[dt_orig.strftime('%Y-%m-%d'), check, tmp]], columns =  ['date','accuracy', 'potential'])]
#                                    )
#                 else : 
#                     acc = pd.concat([acc, pd.DataFrame(
#                         [[dt_orig.strftime('%Y-%m-%d'), check, tmp]], columns =  ['date','accuracy', 'potential'])]
#                                    )

#         acc.to_csv("results/LSTM/accuracy/"+i+".csv")           
#         print(i,acc['accuracy'].mean())
        gc.collect()
#         clear_output(wait=False)
    except Exception as e:
        print("❌ An error occurred:")
        traceback.print_exc()
        gc.collect()
for i in df_main.symbol:    
    print(i)
    run_simulation(i)
    break

CNXFMCG
Epoch 1/400
32/32 [==============================] - 3s 15ms/step - loss: 1.6647 - accuracy: 0.3085
Epoch 2/400
32/32 [==============================] - 0s 13ms/step - loss: 1.4678 - accuracy: 0.4206
Epoch 3/400
32/32 [==============================] - 0s 14ms/step - loss: 1.3335 - accuracy: 0.5248
Epoch 4/400
32/32 [==============================] - 0s 12ms/step - loss: 1.1896 - accuracy: 0.5784
Epoch 5/400
32/32 [==============================] - 0s 15ms/step - loss: 1.0658 - accuracy: 0.6518
Epoch 6/400
32/32 [==============================] - 0s 13ms/step - loss: 0.9302 - accuracy: 0.7262
Epoch 7/400
32/32 [==============================] - 0s 14ms/step - loss: 0.8123 - accuracy: 0.7837
Epoch 8/400
32/32 [==============================] - 0s 13ms/step - loss: 0.6907 - accuracy: 0.8333
Epoch 9/400
32/32 [==============================] - 0s 13ms/step - loss: 0.5815 - accuracy: 0.8780
Epoch 10/400
32/32 [==============================] - 1s 19ms/step - loss: 0.4876 - accuracy